## Homework

### Dataset

In this homework, we'll build a model for predicting if we have an image of a dino or a dragon. For this, we will use the "Dino or Dragon?" dataset that can be downloaded from [Kaggle](https://www.kaggle.com/datasets/agrigorev/dino-or-dragon).

In [1]:
# !wget https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip
# !unzip dino-dragon.zip

In [2]:
#@ IMPORTING LIBRARIES AND DEPENDENCIES:
import numpy as np
import tensorflow as tf
from tensorflow import keras

%matplotlib inline

### Model

In [3]:
def make_model(learning_rate=0.002, momentum=0.8):

    
    inputs = keras.Input(shape=(150, 150, 3))

    conv = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(inputs)
    pooling = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv)
    flatten = keras.layers.Flatten()(pooling)

    ffn = keras.layers.Dense(64, activation='relu')(flatten)
    outputs = keras.layers.Dense(1, activation='sigmoid')(ffn)
    
    model = keras.Model(inputs, outputs)
    
    #########################################

    optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    loss = keras.losses.BinaryCrossentropy()

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

- binary crossentropy
- focal loss
- mean squared error
- categorical crossentropy

Answer: binary crossentropy

### Question 2

What's the total number of parameters of the model? You can use the `summary` method for that. 

- 9215873
- 11215873
- 14215873
- 19215873

In [4]:
model = make_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                             

Answer: 11,215,873

### Generators and Training

In [5]:
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
        './train',
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',
        shuffle=True
)
test_generator = datagen.flow_from_directory(
        './test',
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',
        shuffle=True
)

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [6]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
80/80 [==============================] - 10s 74ms/step - loss: 0.6343 - accuracy: 0.6299 - val_loss: 0.5311 - val_accuracy: 0.8046
Epoch 2/10
80/80 [==============================] - 6s 70ms/step - loss: 0.4800 - accuracy: 0.7880 - val_loss: 0.4283 - val_accuracy: 0.8274
Epoch 3/10
80/80 [==============================] - 6s 70ms/step - loss: 0.3927 - accuracy: 0.8388 - val_loss: 0.3747 - val_accuracy: 0.8350
Epoch 4/10
80/80 [==============================] - 6s 70ms/step - loss: 0.3414 - accuracy: 0.8720 - val_loss: 0.3375 - val_accuracy: 0.8731
Epoch 5/10
80/80 [==============================] - 6s 69ms/step - loss: 0.3080 - accuracy: 0.8808 - val_loss: 0.3232 - val_accuracy: 0.8553
Epoch 6/10
80/80 [==============================] - 6s 71ms/step - loss: 0.2821 - accuracy: 0.8877 - val_loss: 0.3455 - val_accuracy: 0.8452
Epoch 7/10
80/80 [==============================] - 6s 72ms/step - loss: 0.2581 - accuracy: 0.9034 - val_loss: 0.3101 - val_accuracy: 0.8629
Epoch 8/10
8

### Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- 0.90
- 0.20

In [7]:
hist = history.history
np.median(hist['accuracy'])

0.8842534422874451

Answer: 0.90

### Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.11
- 0.66
- 0.99
- 0.33

In [8]:
np.std(hist['loss'])

0.13013788197449283

Answer: 0.11

### Data Augmentation

In [9]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = datagen.flow_from_directory(
        './train',
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',
        shuffle=True
)

Found 1594 images belonging to 2 classes.


### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- 0.37
- 0.97

In [10]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
80/80 [==============================] - 6s 73ms/step - loss: 0.1546 - accuracy: 0.9548 - val_loss: 0.2921 - val_accuracy: 0.8782
Epoch 2/10
80/80 [==============================] - 6s 72ms/step - loss: 0.1418 - accuracy: 0.9523 - val_loss: 0.2879 - val_accuracy: 0.8782
Epoch 3/10
80/80 [==============================] - 6s 70ms/step - loss: 0.1175 - accuracy: 0.9668 - val_loss: 0.3010 - val_accuracy: 0.8655
Epoch 4/10
80/80 [==============================] - 6s 71ms/step - loss: 0.1060 - accuracy: 0.9737 - val_loss: 0.2944 - val_accuracy: 0.8807
Epoch 5/10
80/80 [==============================] - 6s 71ms/step - loss: 0.0821 - accuracy: 0.9831 - val_loss: 0.3060 - val_accuracy: 0.8756
Epoch 6/10
80/80 [==============================] - 6s 72ms/step - loss: 0.0640 - accuracy: 0.9906 - val_loss: 0.3119 - val_accuracy: 0.8680
Epoch 7/10
80/80 [==============================] - 6s 71ms/step - loss: 0.0551 - accuracy: 0.9931 - val_loss: 0.2985 - val_accuracy: 0.8731
Epoch 8/10
80

In [11]:
hist = history.history
np.mean(hist['val_loss'])

0.303537517786026

Answer: 0.37

### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

- 0.84
- 0.54
- 0.44
- 0.24

In [12]:
print(hist['val_accuracy'][5:])
np.mean(hist['val_accuracy'][5:])

[0.8680202960968018, 0.8730964660644531, 0.875634491443634, 0.8883248567581177, 0.8807106614112854]


0.8771573543548584

Answer: 0.84